In [2]:
%load_ext autoreload
%autoreload 2

import scipy.special as sps
import scipy.stats as scstats
import numpy as np

import torch

import matplotlib.pyplot as plt
from matplotlib import patches
from matplotlib import transforms
import daft

import sys
sys.path.append("../../neuroprob")
sys.path.append("../scripts/") # access to scripts


import os
if not os.path.exists('./output'):
    os.makedirs('./output')
    

from neuroprob import utils

import model_utils

import pickle


plt.style.use(['paper.mplstyle'])

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
def render(pgm):
    """
    Wrapper for rendering PGM via daft
    """
    for plate in pgm._plates:
        plate.render(pgm._ctx)

    for edge in pgm._edges:
        edge.render(pgm._ctx)

    for name in pgm._nodes:
        pgm._nodes[name].render(pgm._ctx)



def init_figax(pgm, fig, ax):
    """
    Wrapper for initializing PGM via daft
    """
    pgm._ctx._figure = fig
    ax.axis('off')

    # Set the bounds.
    l0 = pgm._ctx.convert(*pgm._ctx.origin)
    l1 = pgm._ctx.convert(*(pgm._ctx.origin + pgm._ctx.shape))
    ax.set_xlim(l0[0], l1[0])
    ax.set_ylim(l0[1], l1[1])
    ax.set_aspect(1)

    pgm._ctx._ax = ax

### Stochastic process over point process conditionals

In [ ]:
a = np.random.randn(10, 100)*5.+10.0
dt = 0.001

In [ ]:
Tl = 3000
sample_bin = 0.001


l = sample_bin*np.array([30.0])[None, :]
dn = l.shape[1]
v = np.ones(dn)



# generate GP trajectories
kernel_tuples = [('variance', v), 
                 ('RBF', 'euclid', l)]

with torch.no_grad():
    kernel, _, _ = GP.kernels.create_kernel(kernel_tuples, 'softplus', torch.double)

    inp = torch.arange(Tl)[None, None, :, None]*sample_bin
    K = kernel(inp, inp)[0, ...]
    K.view(dn, -1)[:, ::Tl+1] += 1e-6


L = torch.cholesky(K)
mc = 10
xi = np.random.randn(mc, dn, Tl)
x = (L[None, ...] * xi[..., None, :]).sum(-1)



# linear Poisson model
neurons = 100
w_len = dn
GPFA = mdl.parametrics.GLM(w_len, neurons, w_len, 'exp', bias=True)

w = np.random.randn(neurons, w_len)
bias = np.random.randn(neurons)
GPFA.set_params(w, bias)


likelihood = mdl.likelihoods.Poisson(sample_bin, neurons, 'exp')
#likelihood.set_Y(rc_t, batch_size=5000, filter_len=1) 

In [ ]:
plt.plot(x.numpy()[1, 0, :])

In [ ]:
x = np.exp(x)
p = x.numpy()[:, 0, :]*np.exp(-np.cumsum(x.numpy()[:, 0, :], axis=1)*dt) # natural time

In [ ]:
tau = np.arange(Tl)*dt
tau_0 = 0.001
t = tau_0*(np.exp(tau) - 1)
dtau_dt = 1/(t+tau_0)

In [ ]:
plt.plot(tau, p.mean(0).T)
plt.plot(tau[:, None].repeat(p.shape[0], axis=1), p.T, alpha=0.3)

In [ ]:
plt.plot(t, p.mean(0).T*dtau_dt)
plt.plot(t[:, None].repeat(p.shape[0], axis=1), p.T*dtau_dt[:, None], alpha=0.3)

In [ ]:
(p*dt).sum(1)

In [ ]:
neurons = 1
hist_couple = None
shape_t = 5.0*np.ones(neurons)
likelihood = mdl.likelihoods.Gamma(sample_bin, neurons, rate_model.inv_link, shape_t)
#sigma_t = 0.5*np.ones(neurons)
#likelihood = mdl.likelihoods.logNormal(sample_bin, neurons, inv_link, sigma_t)

#mu_t = 5.0*np.ones(neurons)
#likelihood = mdl.likelihoods.invGaussian(neurons, inv_link, mu_t)

#hist_len = 99 # 100 steps of spiketrain, no instantaneous element
#hist_couple = mdl.filters.raised_cosine_bumps(a=1., c=1., phi=phi_h, w=w_h, timesteps=hist_len)
#likelihood = mdl.likelihoods.Bernoulli(neurons, inv_link)

#input_group = mdl.inference.input_group(3, [(None, None, None, 1)]*3)
#input_group.set_XZ(covariates, track_samples, batch_size=track_samples, trials=trials)

#glm = mdl.inference.VI_optimized(input_group, gauss_rate, likelihood)
#glm.to(dev)
    

bb_isi = np.linspace(0.001, 5.0, 100) # ISI evaluate

#gt_field.append(compute_rate(model.rate_model[0], [0])[0])
ISI = [torch.tensor(bb_isi[:, None], device='cpu')]#dev)]
scale = likelihood.shape.data
#scale = torch.exp(-model.likelihood.sigma.data**2/2.)
#scale = 1./model.likelihood.mu
likelihood.trials = 1
p = likelihood.nll(torch.log(scale)*torch.ones((len(bb_isi), 1), device='cpu'), [[ISI[0]*scale]], [0])#.data[:, 0].cpu().numpy()
a = np.exp(-p)

b = likelihood.ISI_dist([0]).prob_dens(bb_isi)

plt.plot(bb_isi, a)
plt.plot(bb_isi, b, 'r--')
plt.show()

### Schematic

In [ ]:
fig = plt.figure(figsize=(8,5)) # plot fits
fig.text(-0.21, 1.16, 'A', transform=ax.transAxes, size=15)

time_bins = I_ext[0].shape[0]
tt = np.arange(time_bins)*dt



widths = [1]
heights = [1, 1, 1, 2]
spec = fig.add_gridspec(ncols=1, nrows=4, width_ratios=widths, height_ratios=heights, 
                        left=0., right=0.7, bottom=0., top=1.0)
